In [ ]:
from datasets import load_dataset
import evaluate
import numpy 
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DataCollatorWithPadding, EarlyStoppingCallback, RobertaForSequenceClassification, RobertaTokenizer, TrainingArguments, Trainer

In [ ]:
# Setting computer for CUDA usage, else CPU usage
def CUDA_CPU():
    if torch.cuda.is_available():
        print("CUDA is available. Using CUDA.")
        return 'cuda'
    else:
        print("CUDA isn't available. Using CPU.")
        return 'cpu'

device = CUDA_CPU()

# Dataset Class 
class DatasetAttr(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        tokenized_input = self.tokenizer(item['Sentence'], padding = "max_length", truncation = True, max_length = 512)
        return {**tokenized_input, "labels": item['Sentiment']}

# Accuracy Metric Setup 
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim = 1).numpy()

    accuracy = evaluate.load("accuracy")
    f1 = evaluate.load("f1")
    precision = evaluate.load("precision")
    recall = evaluate.load("recall")

    results = {
        "accuracy": accuracy.compute(predictions = predictions, references = labels)["accuracy"],
        "f1": f1.compute(predictions = predictions, references = labels, average = "macro")["f1"],
        "precision": precision.compute(predictions = predictions, references = labels, average = "macro", zero_division = 0)["precision"],
        "recall": recall.compute(predictions = predictions, references = labels, average = "macro", zero_division = 0)["recall"],
    }

    return results

In [ ]:
# Training and evaluation function
def train_evaluate():
    try:
        # Loading and Preprocessing Dataset
        dataset = load_dataset('KennNguyenDev/FiQA_Financial_Phrasebank_Combined')
        train_validtest = dataset['train'].train_test_split(test_size = 0.4)
        valid_test = train_validtest['train'].train_test_split(test_size = 0.5)
        train_set, valid_set, test_set = train_validtest['train'], valid_test['train'], valid_test['test']

        # Tokenizer and model initialization
        tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        train_dataset = DatasetAttr(train_set, tokenizer)
        valid_dataset = DatasetAttr(valid_set, tokenizer)
        test_dataset = DatasetAttr(test_set, tokenizer)
        
        config = RobertaForSequenceClassification.from_pretrained('roberta-base').config
        config.num_labels = 3  
        config.hidden_dropout_prob = 0.3
        config.attention_probs_dropout_prob = 0.3
        model = RobertaForSequenceClassification(config)

        # Trainer setup
        training_args = TrainingArguments(
            output_dir = "./results",
            num_train_epochs = 30,
            per_device_train_batch_size = 8,
            per_device_eval_batch_size = 8,
            warmup_steps = 500,
            weight_decay = 0.01,
            evaluation_strategy = "epoch",
            save_strategy = "epoch",
            learning_rate = 5e-05,
            load_best_model_at_end = True,  
            metric_for_best_model = "eval_loss",  
            greater_is_better = False,  
        )

        trainer = Trainer(
            model = model,
            args = training_args,
            train_dataset = train_dataset,
            eval_dataset = valid_dataset,
            compute_metrics = compute_metrics,
            callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)],
        )

        # Training and evaluation
        trainer.train()
        trainer.save_model("./saved_model")

        test_results = trainer.evaluate(test_dataset)
        print(f"Model Over Test Dataset Results: {test_results}")
    
    except Exception as error:
        print(f"An error occurred: {error}")

train_evaluate()